In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from pyproj import Transformer
import folium
import webbrowser

In [ ]:
%matplotlib inline

In [ ]:
#find working folder
dir_path = os.getcwd()

In [ ]:
df = pd.read_csv(dir_path + '\\example\\example_data.csv')
# convert x/y columns to number from text
df[["EASTING", "NORTHING"]] = df[["EASTING", "NORTHING"]].apply(pd.to_numeric)

In [ ]:
def vectorized_convert(df):
    transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
    converted = transformer.transform(df['EASTING'].tolist(), df['NORTHING'].tolist())
    df['lat'] = converted[1]
    df['lon'] = converted[0]
    return df
vec = vectorized_convert(df)

In [ ]:
vec = gpd.GeoDataFrame(vec, geometry=gpd.points_from_xy(vec['lat'], vec['lon']),crs="EPSG:4326")

In [ ]:
# read boundary into dataframe and transform
service_area = gpd.read_file(dir_path + "/data/geospatial/DSFRS_Service_Area.shp")
service_area = service_area.to_crs(epsg=4326)
#transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
#boundary = transformer.transform(boundary['geometry'])

# combine and filter points within boundary
vec = gpd.sjoin(vec, service_area, how="inner", op='within')

In [ ]:
coords = vec[['lat', 'lon']].values
miles = 2
conversion_factor = 0.62137119
kilometers = miles / conversion_factor
kms_per_radian = 6371.0088
epsilon = kilometers / kms_per_radian
# Compute DBSCAN
db = DBSCAN(eps=epsilon, min_samples=12, metric='haversine', algorithm='ball_tree').fit(np.radians(coords))
# Storing the labels formed
labels = db.labels_
# Identifying which points make up the “core points”
core_points = np.zeros_like(labels, dtype = bool)
core_points[db.core_sample_indices_] = True

In [ ]:
# Number of clusters in labels, ignoring noise if present.
num_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
num_noise_ = list(labels).count(-1)
print('Number of clusters: {}'.format(num_clusters_))
print('Number of noise points: {}'.format(num_noise_))
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(coords, labels))

clusters = pd.Series([coords[labels == n] for n in range(num_clusters_)])
print(clusters)

In [ ]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)
#print(centermost_points)
#centermost_points.to_csv("centre.csv", index=False)
#lats, lons = zip(*centermost_points)
#rep_points = pd.DataFrame({'lat':lats, 'lon':lons})
rs = pd.DataFrame(list(centermost_points), columns=["lat", "lon"])

In [ ]:
fig, ax = plt.subplots(figsize=[18, 10])
rs_scatter = ax.scatter(rs['lat'], rs['lon'], c='#99cc99', edgecolor='None', alpha=0.7, s=300)
df_scatter = ax.scatter(vec['lat'], vec['lon'], c='k', alpha=0.9, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
plt.show()

In [ ]:
mask = vec['ERS_TYPE'] == 'DWELLING'
dwl = vec[mask]
rtc = vec[~mask]

In [ ]:
# Build map 
m = folium.Map(location=(50.909614, -3.48), zoom_start=9, zoomSnap=0.1, zoomDelta=0.5, tiles=None, control_scale=True, layer_name = 'Shaking Intensity')
folium.TileLayer('cartodbpositron',name='Greyscale').add_to(m)

# set custom name for base layer
#m = folium.Map((0, 0), tiles=None)
#folium.TileLayer('cartodbpositron', name='my tilelayer').add_to(m)

layer = folium.FeatureGroup(name='Clusters', show=True)
layer2 = folium.FeatureGroup(name='Dwl Fires', show=True)
layer3 = folium.FeatureGroup(name='RTCs', show=True)


# add marker one by one for the clusters
for i in range(0,len(rs)):
   folium.CircleMarker(location=[rs.iloc[i]['lon'], rs.iloc[i]['lat']], radius=12, stroke=True, color='firebrick', weight=3, fill=False,
      fill_color='#ffffff', fillOpacity=1).add_to(layer)

# add marker one by one for dwl
for i in range(0,len(dwl)):
   folium.CircleMarker(location=[dwl.iloc[i]['lon'], dwl.iloc[i]['lat']], radius=4, stroke=False, color='#fffa76', opacity=0.8, weight=2, fill=True,
      fill_color='#fffa76', fillOpacity=1).add_to(layer2)

# add marker one by one for rtc
for i in range(0,len(rtc)):
   folium.CircleMarker(location=[rtc.iloc[i]['lon'], rtc.iloc[i]['lat']], radius=4, stroke=False, color='#f676ff', opacity=0.8, weight=2, fill=True,
      fill_color='#f676ff', fillOpacity=1).add_to(layer3)

# calculate bottom left and top right points and fit the window to those
sw = vec[['lon', 'lat']].min().values.tolist()
ne = vec[['lon', 'lat']].max().values.tolist()
m.fit_bounds([sw, ne])

style = {'color': '#a9a9a9', 'fillColor': '#f5f5f5' }  # 'lineColor': '#ffffbf' blue
folium.GeoJson(service_area, name='Service_Area', style_function = lambda x: style).add_to(m)
layer.add_to(m)
layer2.add_to(m)
layer3.add_to(m)

#folium.GeoJson(service_area).add_to(m)
folium.map.LayerControl('topright', collapsed=True).add_to(m) #, hideSingleBase=True
#folium.LayerControl().add_to(m)
m